# Sensitivity analysis on models governed by stochastic fields :
## Analysis & Validation

- Date: 14.10.20
- Author: K. A. Simady 

##### Aim of the notebook : present the methods already developed as well as a simple example, and compare the already known methods to the ones developed. 
- **Theme :** Sensitivity analysis of models taking as an input scalar random variables, as well as random stochastic fields. 
- **Example :** Model of a beam being supported on both ends and subject to a point force. Five quantities are subject to uncertainties : 
    - The diameter D_ of the section of the beam is varying along a one dimensional field along the axis of the beam. 
    - The Young Modulus E_ is also varying stochastically along the same axis
    - The position of the application point of the force FPos is following a scalar normal law centered on the beams middle point. 
    - The norm of the force vector FNorm is also follwing a scalar normal law centered around 100N 
    - The global densitiy of the beam which is also determined by a scalar normal law centered around the materials real density
- **Method :** Roughly, the method consist in reexpressing the model (which was taking as inputs **Fields** & **Scalars**) as a new model only dependent of a **Scalar vector**. This is made possible through the usage of the **Karhunen-Loeve decomposition**, thanks to which one can freely express a field generated by a stochastic process of known parameters as a vector of scalars. This decomposition is analogous to the **Fourier** decomposition. Once the model is re-expressed a metamodel is built using either krieging or polynomial chaos. Then the sensitivity analysis is done on this metamodel, in hope that it behaves as the real model. This will be tested here.
- **Application :** This method is developed to be used to analyse the model of a aeronautics grade heat exchanger, but this example will not be presented here. 

In [1]:
# Let's first import the necessary modules. 
# Base modules :
import os
from collections.abc import Sequence, Iterable
import numpy as np
import openturns as ot
import pandas as pd
# Phimeca's modules :
#import pythontools as pt # tools created for krieging, pce and other stuff... 
# Own modules : 
import KarhunenLoeveFieldSensitivity as klfs # integration of the codes developed ... 
# Example Model :
import beamExample as MODEL # Could be any model really... 

len vertices is: 101


- The inputs to our model are the 5 uncertain variables, which effects we try to measure. As for the Karhunen-Loeve decomposition the parameters of the stochastic process must be known (scale, amplitude, autocorrelation function) we will first define those. 
- Only two of our 5 inputs are stochastic fields, but we need to also express the other ones as fields. To achieve this, we can think about a scalar as the first value of a constant stochastic_field. 

##### Let's define our processes :
- As only the autocovariance function and the mesh is needed for the **Karhunen Loeve** decomposition, we have no need to neither define our gaussian processes nor to set a trend function. This has some implications :
    - **The trend function was as a constant to represent the mean value of our process. As it is no longer present, in the decomposition, we have to directly add the means inside of our model and consider the inputs as sole perturbations of the mean.**

In [2]:
# Let's first define the mesh, as it is the same for all the processes, random variables and the finite elements of the beam itself. 
dimension = 1
NElem = [99]
mesher = ot.IntervalMesher(NElem)
lowerBound = [0] #mm
upperBound = [1000] #mm
interval = ot.Interval(lowerBound,upperBound)
mesh = mesher.build(interval)
# 100 elements of 10 mm


In [3]:
# First process decomposition : Stochastic young's modulus 
amplitude0 = [210000*.05]*dimension # Mean : 210000 MPaa
scale0 = [300]*dimension 
nu0 = 13/3
model0 = ot.MaternModel(scale0, amplitude0, nu0)

process=ot.GaussianProcess(model0, mesh)

# Karhunen Loeve decomposition of process 
algorithm = ot.KarhunenLoeveP1Algorithm(mesh, model0, 1e-3)
#algorithm.setNbModes(6)
algorithm.run()
resultsE = algorithm.getResult()
resultsE.setName('E_')
resultsE.getCovarianceModel()

class=MaternModel scale=class=Point name=Unnamed dimension=1 values=[300] amplitude=class=Point name=Unnamed dimension=1 values=[10500] nu=4.33333

In [4]:
# Second process decomposition : Stochastic Diameter
amplitude = [10*.05]*dimension # Mean 10 mm
scale = [250]*dimension
nu = 7.4/3
model1 = ot.MaternModel(scale, amplitude, nu)
algorithm = ot.KarhunenLoeveP1Algorithm(mesh, model1, 1e-3)
#algorithm.setNbModes(6)
algorithm.run()
resultsD = algorithm.getResult()
resultsD.setName('D_')

In [5]:
# The two scalar random variables : 
# random variable for the position of the force (mm) 
sigma_f      = 500*.05
RV_Fpos = ot.Normal(500, sigma_f)
RV_Fpos.setName('FPos')
# random variable for the norm of the force (N)
sigma_Fnor    = 100*.05
RV_Fnorm  = ot.Normal(100, sigma_Fnor)
RV_Fnorm.setName('FNorm')

# With the modifs, we can now utilize directly the Distributions

#### Once all the processes are defined, created and decomposed thanks to Karhunen Loeve, we use our newly created class **AggregatedKarhunenLoeveResults**. This class is the link between our reduced normal centered law vector, being one dimensional, and our dimension of arbitrary fields and scalars (so either a-dimensional or multi-dimensional). 

In [6]:
listOfKLRes = [resultsE, resultsD, RV_Fpos, RV_Fnorm]
AggregatedKLRes = klfs.AggregatedKarhunenLoeveResults(listOfKLRes)
AggregatedKLRes.setMean(0, 210000) # At other indices the means are initialized from the distributions
AggregatedKLRes.setMean(1, 10) 
AggregatedKLRes.setLiftWithMean(True)

The mean value of distribution at index 2 of type Normal is not 0.
Distribution recentered and mean added to list of means
Set the "liftWithMean" flag to true if you want to include the mean.
The mean value of distribution at index 3 of type Normal is not 0.
Distribution recentered and mean added to list of means
Set the "liftWithMean" flag to true if you want to include the mean.
The process names are not unique.
Using generic name. 
Old name was E_, new one is X_0
Old name was D_, new one is X_1
Old name was Normal, new one is X_2
Old name was Normal, new one is X_3


###### Depending on how our model was defined, some more steps may be performed if necessary. In our case, to initialize our model, we first have to pass it a mesh from which dimensions it builds its finite element representation, entirely based on the mesh on which the processes where defined. Then we differentiate our model into two functions, one that works on multiple inputs with multiprocessing and one only for single evaluations. **This differntation step is optional, only one of the two functions is needed**
- **The functions as well as the aggregated Karhunen Loeve Results object are passed to a other newly defined class: KarhunenLoeveGeneralizedFunctionWrapper :
    > As this class has access to the Karhunen-Loeve decomposition (which is as said the link between our differnt dimensions) and the model to analyse, this wrapper allows us to get a other view of the model, a view where the model is only dependent of a input vector of values following a centered reduced normal law.

In [7]:
# definition of the model :
_MODEL = MODEL.PureBeam()

# initialization of the function wrapper : 
FUNC = klfs.KarhunenLoeveGeneralizedFunctionWrapper(
                                AggregatedKarhunenLoeveResults = AggregatedKLRes,
                                func        = None, 
                                func_sample = _MODEL.batchEval,
                                n_outputs   = 2) #We have to define the number of elements in the tuple the model returns !!!!!!!!!!!
AggregatedKLRes

class = ComposedKarhunenLoeveResultsAndDistributions| name = Unnamed| Aggregation Order = 4| Threshold = 0.001| Covariance Model 0 = class=MaternModel scale=class=Point name=Unnamed dimension=1 values=[300] amplitude=class=Point name=Unnamed dimension=1 values=[10500] nu=4.33333| Covariance Model 1 = class=MaternModel scale=class=Point name=Unnamed dimension=1 values=[250] amplitude=class=Point name=Unnamed dimension=1 values=[0.5] nu=2.46667| Covariance Model 2 = None| Covariance Model 3 = None| Eigen Value 0 = class=Point name=Unnamed dimension=7 values=[6.25878e+10,3.11591e+10,1.14197e+10,3.55608e+09,1.04551e+09,3.09588e+08,9.56649e+07]| Eigen Value 1 = class=Point name=Unnamed dimension=10 values=[121.91,70.6145,32.8343,13.8746,5.79651,2.50434,1.14062,0.550531,0.281169,0.151298]| Eigen Value 2 = None| Eigen Value 3 = None| Mesh 0 = class=Mesh name=Unnamed dimension=1 vertices=class=Sample name=Unnamed implementation=class=SampleImplementation name=Unnamed size=100 dimension=1 | Mes

#### Now that we have defined our model and input parameters we can begin to design our DOE (Design of experiment). We will generate mutiple DOEs, as we want to compare multiple methods together. 
- **As we know that ALL the components of the vector entering into our wrapped functions are following a centered resuced normal law we have different ways of generating our DOE.**
    - First of all we need 3 types of DOEs :
        - One for the krieging with 100 points : This one will be generated with the LHS sampling method. For this generation we do not need any class in particular. 
            - => For this sample we also need the response (or output) of the finite element model (FUNC) 
        - Two for the calculus of the sobol indices. These ones will be much bigger and will only follow a random sampling method. Only one of these samples will be evaluated on the finite element model. The other one will be evaluated on the krieging metamodel to compare the sobol indices in both cases. 
    - As we want to test the inluence of the different parameters having an influence we will prepaper multiple DOEs per type of DOE. :
        - For the first type (the krieging model):
            - one DOE with 50 points following the LHS sampling method    | SEED : 130875 => CALCULUS OF THE FEM MODEL RESPONSE
            - one DOE with 100 points following the LHS sampling method   | SEED : 409484 => CALCULUS OF THE FEM MODEL RESPONSE
            - one DOE with 200 points following the LHS sampling method   | SEED : 224409 => CALCULUS OF THE FEM MODEL RESPONSE

        - For the second and third type (sobol on the fem model):
            - one DOE with 1000 points following a random sampling method | SEED : 248214 => CALUCLUS OF THE FEM MODEL RESPONSE
            - one DOE with 2000 points following a random sampling method | SEED : 720442 => CALUCLUS OF THE FEM MODEL RESPONSE
            - one DOE with 4000 points following a random sampling method | SEED : 109242 => CALUCLUS OF THE FEM MODEL RESPONSE



In [8]:
import functools
import time
def timer(func):
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        tic = time.perf_counter()
        value = func(*args, **kwargs)
        toc = time.perf_counter()
        elapsed_time = toc - tic
        print(f"Elapsed time: {elapsed_time:0.4f} seconds")
        return value
    return wrapper_timer


# Little helper class for optimized lhs :
@timer
def optimizedLHS(distribution, size, seed):
    ot.RandomGenerator.SetSeed(seed)
    lhs = ot.LHSExperiment(distribution, size, True, True)
    lhs_optimise = ot.SimulatedAnnealingLHS(lhs)
    lhs_sample = lhs_optimise.generate()
    return lhs_sample

@timer
def getSample(distribution, size, seed):
    ot.RandomGenerator.SetSeed(seed)
    sample = distribution.getSample(size)
    sample.setDescription(distribution.getDescription())
    return sample

@timer 
def getSobolExperiment(size, seed, secondOrder = False):
    ot.RandomGenerator.SetSeed(seed)    
    experiment = klfs.KarhunenLoeveSobolIndicesExperiment(AggregatedKLRes, size, secondOrder)
    sobolExp = experiment.generate()
    return sobolExp, experiment



To build our design of experiment, we will need some data about our KL decomposition, namely the order of decomposition and a random normal vector

In [9]:
nModes = AggregatedKLRes.getSizeModes()  # the number of elements in the input vector of our KL wrapped model
randNormVect = ot.ComposedDistribution([ot.Normal()] * nModes)  # 
randNormVect.setDescription(AggregatedKLRes._getModeDescription())
randNormVect

class=ComposedDistribution name=ComposedDistribution dimension=19 copula=class=IndependentCopula name=IndependentCopula dimension=19 marginal[0]=class=Normal name=Normal dimension=1 mean=class=Point name=Unnamed dimension=1 values=[0] sigma=class=Point name=Unnamed dimension=1 values=[1] correlationMatrix=class=CorrelationMatrix dimension=1 implementation=class=MatrixImplementation name=Unnamed rows=1 columns=1 values=[1] marginal[1]=class=Normal name=Normal dimension=1 mean=class=Point name=Unnamed dimension=1 values=[0] sigma=class=Point name=Unnamed dimension=1 values=[1] correlationMatrix=class=CorrelationMatrix dimension=1 implementation=class=MatrixImplementation name=Unnamed rows=1 columns=1 values=[1] marginal[2]=class=Normal name=Normal dimension=1 mean=class=Point name=Unnamed dimension=1 values=[0] sigma=class=Point name=Unnamed dimension=1 values=[1] correlationMatrix=class=CorrelationMatrix dimension=1 implementation=class=MatrixImplementation name=Unnamed rows=1 columns=1 values=[1] marginal[3]=class=Normal name=Normal dimension=1 mean=class=Point name=Unnamed dimension=1 values=[0] sigma=class=Point name=Unnamed dimension=1 values=[1] correlationMatrix=class=CorrelationMatrix dimension=1 implementation=class=MatrixImplementation name=Unnamed rows=1 columns=1 values=[1] marginal[4]=class=Normal name=Normal dimension=1 mean=class=Point name=Unnamed dimension=1 values=[0] sigma=class=Point name=Unnamed dimension=1 values=[1] correlationMatrix=class=CorrelationMatrix dimension=1 implementation=class=MatrixImplementation name=Unnamed rows=1 columns=1 values=[1] marginal[5]=class=Normal name=Normal dimension=1 mean=class=Point name=Unnamed dimension=1 values=[0] sigma=class=Point name=Unnamed dimension=1 values=[1] correlationMatrix=class=CorrelationMatrix dimension=1 implementation=class=MatrixImplementation name=Unnamed rows=1 columns=1 values=[1] marginal[6]=class=Normal name=Normal dimension=1 mean=class=Point name=Unnamed dimension=1 values=[0] sigma=class=Point name=Unnamed dimension=1 values=[1] correlationMatrix=class=CorrelationMatrix dimension=1 implementation=class=MatrixImplementation name=Unnamed rows=1 columns=1 values=[1] marginal[7]=class=Normal name=Normal dimension=1 mean=class=Point name=Unnamed dimension=1 values=[0] sigma=class=Point name=Unnamed dimension=1 values=[1] correlationMatrix=class=CorrelationMatrix dimension=1 implementation=class=MatrixImplementation name=Unnamed rows=1 columns=1 values=[1] marginal[8]=class=Normal name=Normal dimension=1 mean=class=Point name=Unnamed dimension=1 values=[0] sigma=class=Point name=Unnamed dimension=1 values=[1] correlationMatrix=class=CorrelationMatrix dimension=1 implementation=class=MatrixImplementation name=Unnamed rows=1 columns=1 values=[1] marginal[9]=class=Normal name=Normal dimension=1 mean=class=Point name=Unnamed dimension=1 values=[0] sigma=class=Point name=Unnamed dimension=1 values=[1] correlationMatrix=class=CorrelationMatrix dimension=1 implementation=class=MatrixImplementation name=Unnamed rows=1 columns=1 values=[1] marginal[10]=class=Normal name=Normal dimension=1 mean=class=Point name=Unnamed dimension=1 values=[0] sigma=class=Point name=Unnamed dimension=1 values=[1] correlationMatrix=class=CorrelationMatrix dimension=1 implementation=class=MatrixImplementation name=Unnamed rows=1 columns=1 values=[1] marginal[11]=class=Normal name=Normal dimension=1 mean=class=Point name=Unnamed dimension=1 values=[0] sigma=class=Point name=Unnamed dimension=1 values=[1] correlationMatrix=class=CorrelationMatrix dimension=1 implementation=class=MatrixImplementation name=Unnamed rows=1 columns=1 values=[1] marginal[12]=class=Normal name=Normal dimension=1 mean=class=Point name=Unnamed dimension=1 values=[0] sigma=class=Point name=Unnamed dimension=1 values=[1] correlationMatrix=class=CorrelationMatrix dimension=1 implementation=class=MatrixImplementation name=Unnamed rows=1 columns=1 values=[1] marginal[13]=class=Normal name=Normal dimension=1 mean=

In [10]:
a,b=getSobolExperiment(10,1584)
X = a 
X_A = X[:10,:]
X_An = np.array(X_A)
Y_Bn = np.array(X[10:20])
X_n = np.array(X)
Y = np.zeros(X_n.shape)
for i in range(int(X_n.shape[0]/10)):
    Y[i*10:(i+1)*10, :] = (X_n[i*10:(i+1)*10, :]==Y_Bn)
u=ot.Sample(Y)
np.argwhere(u)

Samples A and B of size 10 and dimension 19
Experiment of size 60 and dimension 19
Elapsed time: 0.0012 seconds


array([[10,  0],
       [10,  1],
       [10,  2],
       [10,  3],
       [10,  4],
       [10,  5],
       [10,  6],
       [10,  7],
       [10,  8],
       [10,  9],
       [10, 10],
       [10, 11],
       [10, 12],
       [10, 13],
       [10, 14],
       [10, 15],
       [10, 16],
       [10, 17],
       [10, 18],
       [11,  0],
       [11,  1],
       [11,  2],
       [11,  3],
       [11,  4],
       [11,  5],
       [11,  6],
       [11,  7],
       [11,  8],
       [11,  9],
       [11, 10],
       [11, 11],
       [11, 12],
       [11, 13],
       [11, 14],
       [11, 15],
       [11, 16],
       [11, 17],
       [11, 18],
       [12,  0],
       [12,  1],
       [12,  2],
       [12,  3],
       [12,  4],
       [12,  5],
       [12,  6],
       [12,  7],
       [12,  8],
       [12,  9],
       [12, 10],
       [12, 11],
       [12, 12],
       [12, 13],
       [12, 14],
       [12, 15],
       [12, 16],
       [12, 17],
       [12, 18],
       [13,  0],
       [13,  1

In [11]:
kg_doe50_130875   = optimizedLHS(randNormVect, 50, 130875)
kg_doe100_409484  = optimizedLHS(randNormVect, 100, 409484)
kg_doe200_224409  = optimizedLHS(randNormVect, 200, 224409)
print('First DOE type done\n')

si_doe1000_248214, expe_doe1000 = getSobolExperiment(1000, 248214)
si_doe2000_720442, expe_doe2000 = getSobolExperiment(2000, 720442)
si_doe4000_109242, expe_doe4000 = getSobolExperiment(4000, 109242)
si_doe8000_439038, expe_doe8000 = getSobolExperiment(8000, 439038)

si_sec_doe2000_8291001288, expe_doe2000_sec = getSobolExperiment(2000, 8291001288, True)

print('Sobol Samples OK.')

valid_doe200_502849 = optimizedLHS(randNormVect, 200, 502849)

Elapsed time: 0.1262 seconds
Elapsed time: 0.4765 seconds
Elapsed time: 1.9216 seconds
First DOE type done

Samples A and B of size 1000 and dimension 19
Experiment of size 6000 and dimension 19
Elapsed time: 0.0062 seconds
Samples A and B of size 2000 and dimension 19
Experiment of size 12000 and dimension 19
Elapsed time: 0.0122 seconds
Samples A and B of size 4000 and dimension 19
Experiment of size 24000 and dimension 19
Elapsed time: 0.0244 seconds
Samples A and B of size 8000 and dimension 19
Experiment of size 48000 and dimension 19
Elapsed time: 0.0479 seconds
Generating samples for the second order indices
Samples A and B of size 2000 and dimension 19
Experiment for second order generated
Experiment of size 20000 and dimension 19
Elapsed time: 0.0180 seconds
Sobol Samples OK.
Elapsed time: 1.9018 seconds


In [12]:
si_doe4000_1092445, expe_doe4000_2 = getSobolExperiment(10000, 1092445)
si_doe50000_3846345, expe_doe50000_3846345 = getSobolExperiment(50000, 3846345)

Samples A and B of size 10000 and dimension 19
Experiment of size 60000 and dimension 19
Elapsed time: 0.0646 seconds
Samples A and B of size 50000 and dimension 19
Experiment of size 300000 and dimension 19
Elapsed time: 0.3268 seconds


In [13]:
si_doe50000_3846345

class=Sample name=Unnamed implementation=class=SampleImplementation name=Unnamed size=300000 dimension=19 description=[X_0_0,X_0_1,X_0_2,...,X_1_9,X_2_0,X_3_0] data=[[0.561322,0.11925,0.671032,...,0.53983,0.950291,0.185707],[1.51427,-0.733568,-0.636825,...,0.0771509,1.39343,-1.85255],[0.121974,-1.09726,0.00593211,...,0.399863,-1.29252,-1.04418],...,[-1.83527,0.625017,0.999093,...,-0.192539,-1.36254,-0.00775361],[-0.470572,-1.54057,-0.909488,...,2.07089,1.68804,-0.417928],[0.810585,1.16692,-0.0182286,...,-0.248038,-0.70419,-0.858629]]

In [14]:
test = FUNC(kg_doe100_409484)

Lifting as process sample
field E shape (100, 100)
var_Fnor shape (100,)
field_E 210055.2129089407 field_D 10.002162804359951 var_Fpos 499.9458882511429 var_Fnor 100.02508378644619


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done  96 out of 100 | elapsed:    5.1s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    5.2s finished
/home/motherloadubu/anaconda3/envs/stochastic_field_env/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple o

shape deflection:  (100, 101)  should be [N,10X] something
deflection std deviation  3.7359493488235063
Using the batch evaluation function. Assumes that the outputs are in the
same order than for the single evaluation function. This one should only
return ProcessSamples, Samples, Lists or numpy arrays.
Element is iterable, assumes that first dimension is size of sample
Shape is (100, 100) and dtype is <class 'numpy.float64'>
Element 0 of the output tuple returns process samples of dimension 1
Element is iterable, assumes that first dimension is size of sample
Shape is (1, 100) and dtype is <class 'numpy.float64'>
Element 1 of the output tuple returns process samples of dimension 1


In [15]:
#resp2 = FUNC(si_doe4000_1092445)
# resp_doe50000_3846345 = FUNC(si_doe50000_3846345)

Lifting as process sample
field E shape (300000, 100)
var_Fnor shape (300000,)
field_E 209972.84936860687 field_D 9.997287115567817 var_Fpos 499.90313545148723 var_Fnor 100.03380949052938


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1574s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done  74 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 122 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 148 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 178 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done 208 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Done 242 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done 276 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-1)]: Done 314 tas

[Parallel(n_jobs=-1)]: Done 17908 tasks      | elapsed:  7.3min
[Parallel(n_jobs=-1)]: Done 18178 tasks      | elapsed:  7.4min
[Parallel(n_jobs=-1)]: Done 18448 tasks      | elapsed:  7.5min
[Parallel(n_jobs=-1)]: Done 18722 tasks      | elapsed:  7.7min
[Parallel(n_jobs=-1)]: Done 18996 tasks      | elapsed:  7.8min
[Parallel(n_jobs=-1)]: Done 19274 tasks      | elapsed:  7.9min
[Parallel(n_jobs=-1)]: Done 19552 tasks      | elapsed:  8.0min
[Parallel(n_jobs=-1)]: Done 19834 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 20116 tasks      | elapsed:  8.3min
[Parallel(n_jobs=-1)]: Done 20402 tasks      | elapsed:  8.4min
[Parallel(n_jobs=-1)]: Done 20688 tasks      | elapsed:  8.5min
[Parallel(n_jobs=-1)]: Done 20978 tasks      | elapsed:  8.7min
[Parallel(n_jobs=-1)]: Done 21268 tasks      | elapsed:  8.8min
[Parallel(n_jobs=-1)]: Done 21562 tasks      | elapsed:  8.9min
[Parallel(n_jobs=-1)]: Done 21856 tasks      | elapsed:  9.1min
[Parallel(n_jobs=-1)]: Done 22154 tasks 

[Parallel(n_jobs=-1)]: Done 69122 tasks      | elapsed: 29.6min
[Parallel(n_jobs=-1)]: Done 69648 tasks      | elapsed: 29.8min
[Parallel(n_jobs=-1)]: Done 70178 tasks      | elapsed: 30.0min
[Parallel(n_jobs=-1)]: Done 70708 tasks      | elapsed: 30.2min
[Parallel(n_jobs=-1)]: Done 71242 tasks      | elapsed: 30.5min
[Parallel(n_jobs=-1)]: Done 71776 tasks      | elapsed: 30.7min
[Parallel(n_jobs=-1)]: Done 72314 tasks      | elapsed: 30.9min
[Parallel(n_jobs=-1)]: Done 72852 tasks      | elapsed: 31.1min
[Parallel(n_jobs=-1)]: Done 73394 tasks      | elapsed: 31.4min
[Parallel(n_jobs=-1)]: Done 73936 tasks      | elapsed: 31.6min
[Parallel(n_jobs=-1)]: Done 74482 tasks      | elapsed: 31.8min
[Parallel(n_jobs=-1)]: Done 75028 tasks      | elapsed: 32.0min
[Parallel(n_jobs=-1)]: Done 75578 tasks      | elapsed: 32.3min
[Parallel(n_jobs=-1)]: Done 76128 tasks      | elapsed: 32.5min
[Parallel(n_jobs=-1)]: Done 76682 tasks      | elapsed: 32.7min
[Parallel(n_jobs=-1)]: Done 77236 tasks 

[Parallel(n_jobs=-1)]: Done 152052 tasks      | elapsed: 63.4min
[Parallel(n_jobs=-1)]: Done 152834 tasks      | elapsed: 63.7min
[Parallel(n_jobs=-1)]: Done 153616 tasks      | elapsed: 64.0min
[Parallel(n_jobs=-1)]: Done 154402 tasks      | elapsed: 64.3min
[Parallel(n_jobs=-1)]: Done 155188 tasks      | elapsed: 64.6min
[Parallel(n_jobs=-1)]: Done 155978 tasks      | elapsed: 64.9min
[Parallel(n_jobs=-1)]: Done 156768 tasks      | elapsed: 65.2min
[Parallel(n_jobs=-1)]: Done 157562 tasks      | elapsed: 65.6min
[Parallel(n_jobs=-1)]: Done 158356 tasks      | elapsed: 65.9min
[Parallel(n_jobs=-1)]: Done 159154 tasks      | elapsed: 66.2min
[Parallel(n_jobs=-1)]: Done 159952 tasks      | elapsed: 66.5min
[Parallel(n_jobs=-1)]: Done 160754 tasks      | elapsed: 66.8min
[Parallel(n_jobs=-1)]: Done 161556 tasks      | elapsed: 67.2min
[Parallel(n_jobs=-1)]: Done 162362 tasks      | elapsed: 67.5min
[Parallel(n_jobs=-1)]: Done 163168 tasks      | elapsed: 67.8min
[Parallel(n_jobs=-1)]: Do

[Parallel(n_jobs=-1)]: Done 266208 tasks      | elapsed: 111.8min
[Parallel(n_jobs=-1)]: Done 267242 tasks      | elapsed: 112.3min
[Parallel(n_jobs=-1)]: Done 268276 tasks      | elapsed: 112.8min
[Parallel(n_jobs=-1)]: Done 269314 tasks      | elapsed: 113.2min
[Parallel(n_jobs=-1)]: Done 270352 tasks      | elapsed: 113.7min
[Parallel(n_jobs=-1)]: Done 271394 tasks      | elapsed: 114.2min
[Parallel(n_jobs=-1)]: Done 272436 tasks      | elapsed: 114.7min
[Parallel(n_jobs=-1)]: Done 273482 tasks      | elapsed: 115.2min
[Parallel(n_jobs=-1)]: Done 274528 tasks      | elapsed: 115.7min
[Parallel(n_jobs=-1)]: Done 275578 tasks      | elapsed: 116.2min
[Parallel(n_jobs=-1)]: Done 276628 tasks      | elapsed: 116.7min
[Parallel(n_jobs=-1)]: Done 277682 tasks      | elapsed: 117.2min
[Parallel(n_jobs=-1)]: Done 278736 tasks      | elapsed: 117.7min
[Parallel(n_jobs=-1)]: Done 279794 tasks      | elapsed: 118.2min
[Parallel(n_jobs=-1)]: Done 280852 tasks      | elapsed: 118.7min
[Parallel(

shape deflection:  (300000, 101)  should be [N,10X] something
deflection std deviation  3.827450701074076
Using the batch evaluation function. Assumes that the outputs are in the
same order than for the single evaluation function. This one should only
return ProcessSamples, Samples, Lists or numpy arrays.
Element is iterable, assumes that first dimension is size of sample
Shape is (300000, 100) and dtype is <class 'numpy.float64'>
Element 0 of the output tuple returns process samples of dimension 1
Element is iterable, assumes that first dimension is size of sample
Shape is (1, 300000) and dtype is <class 'numpy.float64'>
Element 1 of the output tuple returns process samples of dimension 1


In [16]:
vonMises = ot.Sample(np.array(np.stack([np.squeeze(np.asarray(resp2[0][i])) for i in range(len(resp2[0]))]))) 
maxDefl = resp2[1][0]
si_doe4000_1092445.exportToCSVFile(os.path.join('./','inp_10000'+'.csv'), ';')
vonMises.exportToCSVFile(os.path.join('./','out_10000'+'_VM'+'.csv'), ';')
maxDefl.exportToCSVFile(os.path.join('./','out_10000'+'_MD'+'.csv'), ';')

NameError: name 'resp2' is not defined

In [16]:
vonMises_2 = ot.Sample(np.array(np.stack([np.squeeze(np.asarray(resp_doe50000_3846345[0][i])) for i in range(len(resp_doe50000_3846345[0]))]))) 
maxDefl_2 = resp_doe50000_3846345[1][0]
si_doe4000_1092445.exportToCSVFile(os.path.join('./','inp_50000'+'.csv'), ';')
vonMises_2.exportToCSVFile(os.path.join('./','out_50000'+'_VM'+'.csv'), ';')
maxDefl_2.exportToCSVFile(os.path.join('./','out_50000'+'_MD'+'.csv'), ';')

#### Now that we have defined our design of expriments, we're going to evaluate the model on them, to get it's response. Then we're going to all save into csv files in  a folder, and use these as a basis for our analysis

In [ ]:
DOE_names      = ['kg_doe50_130875', 'kg_doe100_409484', 'kg_doe200_224409', 'si_doe1000_248214', 'si_doe2000_720442', 
                  'val_doe200_502849', 'si_sec_doe2000_8291001288' , 'si_doe4000_109242', 'si_doe8000_439038']
DOE_resp_names = ['kg_doe50_130875_resp', 'kg_doe100_409484_resp', 'kg_doe200_224409_resp', 'si_doe1000_248214_resp', 'si_doe2000_720442_resp', 
                  'val_doe200_502849_resp', 'si_sec_doe2000_8291001288_resp', 'si_doe4000_109242_resp', 'si_doe8000_439038_resp']

In [ ]:
l_DOEs = [kg_doe50_130875,kg_doe100_409484,kg_doe200_224409,si_doe1000_248214,si_doe2000_720442, valid_doe200_502849, si_sec_doe2000_8291001288, si_doe4000_109242, si_doe8000_439038] 

In [ ]:
folder = './DOE_RESP2'
for i, doe in enumerate(l_DOEs):
    if not os.path.isfile(os.path.join(folder,DOE_names[i]+'.csv')):
        resp = FUNC(doe)
        vonMises = ot.Sample(np.array(np.stack([np.squeeze(np.asarray(resp[0][i])) for i in range(len(resp[0]))]))) 
        maxDefl = resp[1]
        doe.exportToCSVFile(os.path.join(folder,DOE_names[i]+'.csv'), ';')
        vonMises.exportToCSVFile(os.path.join(folder,DOE_resp_names[i]+'_VM'+'.csv'), ';')
        maxDefl.exportToCSVFile(os.path.join(folder,DOE_resp_names[i]+'_MD'+'.csv'), ';')